In [ ]:
import numpy as np
import pandas as pd

# 3.5. Tratamiento de series temporales I.

- Uno de los usos más relevantes de Pandas en entorno financiero son las series de datos temporales. 
- Trabajar con fechas y con series en el tiempo es muy intuitivo.

## Dates y times nativos de Python: ``datetime`` and ``dateutil``

- Los objetos básicos de Python para trabajar con fechas y horas se encuentran en el módulo ``datetime``.
- El módulo ``dateutil`` se pueden utilizar para un gran número de funcionalidades.
- Puedes construir una fecha manualmente usando el tipo ``datetime``:

In [ ]:
from datetime import datetime

In [ ]:
datetime(year=2015, month=7, day=4)

- El día de hoy:

In [ ]:
now = datetime.now()
now

In [ ]:
now.year, now.month, now.day

- Diferencias entre días

In [ ]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

In [ ]:
delta.days

In [ ]:
delta.seconds

- Podemos sumar y restar periodos:

In [ ]:
from datetime import timedelta
start = datetime(2011, 1, 7)
start + timedelta(12)
start - 2 * timedelta(12)

-  Conversión de Datime a Strings

In [ ]:
stamp = datetime(2011, 1, 3)

In [ ]:
str(stamp)

In [ ]:
stamp.strftime('%Y-%m-%d')

- También al revés:

In [ ]:
value = '2011-01-03'
datetime.strptime(value, '%Y-%m-%d')

In [ ]:
datestrs = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

- Usando el módulo ``dateutil``, puedes parsear fechas de una gran variedad de formatos desde un string:

In [ ]:
from dateutil.parser import parse

In [ ]:
parse('2011-01-03')

In [ ]:
parse('Jan 31, 1997 10:45 PM')

In [ ]:
parse('6/12/2011', dayfirst=True)


- El poder de  ``datetime`` y ``dateutil`` viene dado por su flexibilidad y sintaxis fácil.
- Se pueden usar estos objetos y sus métodos para llevar a cabo casi cualquier operación. 
- Su limitación viene al trabajar con vectores grandes: las listas de objetos datetime de Python son menos eficiente comparado con pandas o numpy

## Pandas para time series

### Estructuras de tiempo 

- Pandas tiene los objetos ``Timestamp`` que combinan la facilidad de  ``datetime`` y ``dateutil`` con una implementación vectorizada eficiente.
- Usando los objetos ``Timestamp``, pandas construye un ``DatetimeIndex``  para ser usado como índice en ``Series`` o ``DataFrame``.
- Las clases de Pandas para series temporales son:
 * ``Timestamp``: reemplazo de ``datetime``, basado en el sistema más eficiente de ``numpy.datetime64``. La estructura del índice asociada será ``DatetimeIndex``.
 * ``Period``: para periodos de tiempo. La estructura del índice asociado es ``PeriodIndex``.
 * ``Timedelta``: para variación de tiempo o duración (más eficiente que ``datetime.timedelta``). La estructura del índice asociado es ``TimedeltaIndex``.
- Los tipos más básicos son ``Timestamp`` y ``DatetimeIndex``.
- La forma más común de crear objetos de este tipo es usando la función ``pd.to_datetime()`` .
- Puede parsear una gran variedad de formatos. 
- Pasar una fecha a esta función produce un ``Timestamp``; pasar una serie de fechas por defecto devuelve un ``DatetimeIndex``:

In [ ]:
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
dates

- Cualquier ``DatetimeIndex`` puede convertirse en un ``PeriodIndex`` con la función ``to_period()``, añadiendo un código de frecuencia ( ``'D'`` = diario) :

In [ ]:
dates.to_period('M')

In [ ]:
dates.to_period('Y')

- Un ``TimedeltaIndex`` se crea, por ejemplo, cuando restamos una fecha de otra:

In [ ]:
dates - dates[0]

- Normalmente el índice como fecha

In [ ]:
dates = [pd.Timestamp(2011, 1, 2), pd.Timestamp(2011, 1, 5),
         pd.Timestamp(2011, 1, 7), pd.Timestamp(2011, 1, 8),
         pd.Timestamp(2011, 1, 10), pd.Timestamp(2011, 1, 12)]
ts = pd.Series(np.random.randn(6), index=dates)
ts

In [ ]:
ts.index

### Indexación, seleccíon y subsección

- Las herramientas de series temporales de Pandas se vuelven realmente útiles al utilizar *indexación*.

In [ ]:
dates = pd.date_range('2015-07-25', periods=15, freq='B')
data = pd.DataFrame({'close':[10,12,14,15,15,19, 20,17, 15, 14, 12,13,13,14,10]}, 
                    index=dates)
data

- Con el índice como DatetimeIndex se puede hacer uso de cualquier sistema de indexación.
- Pasando valores que pueden ser convertidos en fechas:

In [ ]:
data['2015-07-30':'2015-08-4']

- Hay operaciones de indexación adicionales disponibles sólo para fechas.
- Como pasar un año para obtener los datos de ese año:

In [ ]:
data['2015']

In [ ]:
data['2015-07']

## Rangos de fechas: ``pd.date_range()``

- Igual que las funciones ``range()`` (nativo) y ``np.arange()`` (NumPy) que generan una secuencia tomando un punto de inicio, un punto de fin y un paso, las funciones ``pd.date_range()`` (para fechas) y ``pd.period_range()`` (para duraciones) funcionan de forma similar.
- Para crear un vector de fechas, se usa la función 'date_range'. El uso básico es:
 - `date_range(beginning_date, ending_date, freq)`, que devolvería una serie temporal diario entre ambas fechas (incluidas),
 - `date_range(beginning_date, periods=periods, freq)` que devolvería un rango de fechas desde beginning_date con tantas fechas como se especifiquen en 'periods'. 


In [ ]:
pd.date_range('2015-07-03', '2015-07-10')

- A través de la definición del número de periodos:

In [ ]:
pd.date_range('2015-07-03', periods=8)

- Se puede introducir un atributo ``freq`` para cambiar la frecuencia de las fechas de diaria (``D``) a otras diferentes.
- Se dispone de las sigueintes frecuencias:

| Code   | Description         |
|--------|---------------------|
| ``D``  | Calendar day        |
| ``B``  | Business day        |
| ``W``  | Weekly              |       
| ``M``  | Month end           | 
|``BM`` | Business month end   |
| ``Q``  | Quarter end         |
| ``BQ`` | Business quarter end|
| ``A``  | Year end            | 
|``BA`` | Business year end    |
| ``H``  | Hours               |
|``BH`` | Business hours       |
| ``T`` o ``min`` | Minutes             |       
| ``S``  | Seconds             |       
| ``L``  | Milliseonds         |       
| ``U``  | Microseconds        |     
| ``N``  | nanoseconds         |       

- Las frecuencias mensual, trimestral y anual se proporcionan como fecha al final del periodo especificado.
- Añadiendo una 'S' al final de cada una de estas, la salida será al inicio del periodo.

| Code    | Description            |
|---------|------------------------|
| ``MS``  | Month start            |
| ``QS``  | Quarter start          |
| ``AS``  | Year start             |
|``BMS``  | Business month start   |
|``BQS``  | Business quarter start |
|``BAS``  | Business year start    |

- Además se puede cambiar el mes usado en frecuencias trimestrales o anuales añadiendo un código de tres letras al final:
 * ``Q-JAN``, ``BQ-FEB``, ``QS-MAR``, ``BQS-APR``, etc.
 * ``A-JAN``, ``BA-FEB``, ``AS-MAR``, ``BAS-APR``, etc.

In [ ]:
pd.date_range('2015-07-03', periods=8, freq='H')

In [ ]:
pd.date_range('2015-07-03', '2015-07-04', freq='H')

- Se puede combinar con números para otras frecuencias. 
- Por ejemplo para frecuencias de 2 horas y 30 minutos, se puede combinar la hora (``H``) y los minutos (``T``):

In [ ]:
pd.date_range('2015-07-03', periods=8, freq='2H30min')

- Tamien para un día concreto de la semana del mes.
- Como por ejemplo el 3 viernes de cada mes:

In [ ]:
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI')
list(rng)

- Para crear secuencias de ``Period`` o ``Timedelta``, se puede usar igualmente ``pd.period_range()`` y ``pd.timedelta_range()``.
- Por ejemplo, para periodos mensuales:

In [ ]:
pd.period_range('2015-07', periods=8, freq='M')

- Y para secuencias de duración incremental de hora en hora:

In [ ]:
pd.timedelta_range(0, periods=10, freq='H')